# Compare Matched Trip to Shortest/Modeled Trip
This notebook is for cross-referencing the modeled route against the chosen route. The links attributes and turns are displayed to ensure routing is functioning properly.

In [ ]:
from pathlib import Path
import time
import pandas as pd
import geopandas as gpd
import numpy as np
import pickle
import networkx as nx
from stochopy.optimize import minimize
import stochastic_optimization
from tqdm import tqdm
import similaritymeasures
import random

from shapely.ops import LineString, MultiLineString


import sys
sys.path.insert(0,str(Path.cwd().parent))
import file_structure_setup
config = file_structure_setup.filepaths()
from network.src import modeling_turns
import speedfactor

## Network Import

In [ ]:
#export calibration network
with (config['calibration_fp']/"calibration_network.pkl").open('rb') as fh:
    links, turns = pickle.load(fh)
#turn_G = modeling_turns.make_turn_graph(turns)
links.set_index(['linkid','reverse_link'],inplace=True)
turns.set_index(['source_linkid','source_reverse_link','target_linkid','target_reverse_link'],inplace=True)

turns = turns[['source_B','turn_type','signalized','unsig_major_road_crossing']]

In [ ]:
nodes = gpd.read_file(config['network_fp']/'final_network.gpkg',layer='nodes')
nodes.set_index('N',inplace=True)

In [ ]:
#add nodes geo to turns
turns = turns.merge(nodes['geometry'],left_on='source_B',right_index=True)
turns = gpd.GeoDataFrame(turns,crs=links.crs)

## import route results


In [ ]:
with (config['calibration_fp']/"modeled_results.pkl").open('rb') as fh:
    modeled_results = pickle.load(fh)

In [ ]:
#import 
matched_summary = gpd.read_file(config['calibration_fp']/"route_attributes.gpkg",layer='matched')
modeled_summary = gpd.read_file(config['calibration_fp']/"route_attributes.gpkg",layer='modeled')

In [ ]:
turn_dict = {
    True: "yellow",
    False: "black"
}


In [ ]:
#give each unique combo of values a random color
links.columns

In [ ]:
import random
tripid = random.choice(list(modeled_results.keys()))
tripid

In [ ]:
links_fields = ['name','highway','AADT', 'truck_pct', 'speed', 'length_ft',
       'lanes', 'link_type_new', 'motorized', 'facility_fwd','ascent_ft',
       'ascent_grade_%', 'above_4', 'mixed_traffic_no_facil',
       'mixed_traffic_w_facil']

In [ ]:
links.columns

In [ ]:
import folium

#tripid = 71

route_attribute_cols = ['length_mi']

# Create copies to prevent alteration
matched_gdf = matched_summary.copy()
modeled_gdf = modeled_summary.copy()

# Subset data to relevant trip
matched_gdf = matched_gdf[matched_gdf['tripid']==tripid]
modeled_gdf = modeled_gdf[modeled_gdf['tripid']==tripid]

# Create a Folium map centered around the mean of the matched route
minx, miny, maxx, maxy = matched_gdf.to_crs(epsg='4326').total_bounds
x_mean = (maxx - minx) / 2 + minx
y_mean = (maxy - miny) / 2 + miny
center = [y_mean,x_mean]
m = folium.Map(location=center, zoom_start=14, tiles="cartodbpositron")

# Add GeoJSON data to FeatureGroups
folium.GeoJson(matched_gdf.to_crs(epsg='4326').to_json(),
                name='Matched',
                tooltip=folium.GeoJsonTooltip(fields=route_attribute_cols),
                style_function=lambda x: {'color': 'grey','opacity':0.5}).add_to(m)

folium.GeoJson(modeled_gdf.to_crs(epsg='4326').to_json(),
                name='Modeled',
                tooltip=folium.GeoJsonTooltip(fields=route_attribute_cols),
                style_function=lambda x: {'color': 'orange','opacity':0.5}).add_to(m)

### MATCHED ###
matched_edges = [list(x) for x in modeled_results[tripid]['matched_edges'].values]
matched_turns = [(matched_edges[i][0],matched_edges[i][1],matched_edges[i+1][0],matched_edges[i+1][1]) for i in range(0,len(matched_edges)-1)]
matched_turns = turns.loc[matched_turns]
matched_turn_type = matched_turns.loc[
    matched_turns['turn_type'].notna() &
    ((matched_turns['turn_type']!='straight') | (matched_turns['signalized']==True) | (matched_turns['unsig_major_road_crossing']==True))
    ]
folium.GeoJson(
    links.loc[matched_edges].to_crs(epsg='4326').to_json(),
    name='Matched Edges',
    show=False,
    popup=folium.GeoJsonPopup(fields=links_fields),
    highlight_function=lambda x: {"color": 'yellow'},
    style_function=lambda x: {'color':'gray'}
).add_to(m)
folium.GeoJson(
    matched_turn_type.to_crs(epsg='4326').to_json(),
    name="Matched Turn Types",
    show=False,
    tooltip=folium.GeoJsonTooltip(fields=['turn_type','signalized','unsig_major_road_crossing']),
    marker=folium.Circle(radius=30, fill_color="orange", fill_opacity=0.5, color="black", weight=0),
    style_function=lambda x: {
        'fillColor':turn_dict[x['properties']['signalized']]
    }
).add_to(m)

### MODELED ####
modeled_edges = [tuple(x) for x in modeled_results[tripid]['modeled_edges'].values]
modeled_turns = [(modeled_edges[i][0],modeled_edges[i][1],modeled_edges[i+1][0],modeled_edges[i+1][1]) for i in range(0,len(modeled_edges)-1)]
modeled_turns = turns.loc[modeled_turns]
modeled_turn_type = modeled_turns.loc[
    modeled_turns['turn_type'].notna() &
    ((modeled_turns['turn_type']!='straight') | (modeled_turns['signalized']==True) | (matched_turns['unsig_major_road_crossing']==True))
    ]
folium.GeoJson(
    links.loc[modeled_edges].to_crs(epsg='4326').to_json(),
    name='Modeled Edges',
    show=False,
    popup=folium.GeoJsonPopup(fields=links_fields),
    highlight_function=lambda x: {"color": 'yellow'},
    style_function=lambda x: {'color':'orange'}
).add_to(m)
folium.GeoJson(
    modeled_turn_type.to_crs(epsg='4326').to_json(),
    name="Modeled Turn Types",
    show=False,
    tooltip=folium.GeoJsonTooltip(fields=['turn_type','signalized','unsig_major_road_crossing']),
    marker=folium.Circle(radius=30, fill_color="orange", fill_opacity=0.5, color="black", weight=0),
    style_function=lambda x: {
        'fillColor':turn_dict[x['properties']['signalized']]
    }
).add_to(m)



# Get the start and end points
start_node = modeled_results[tripid]['origin_node']
start_node = nodes.to_crs('epsg:4326').loc[start_node,'geometry']
folium.Circle(location=[start_node.y, start_node.x],color='green').add_to(m)

end_node = modeled_results[tripid]['destination_node']
end_node = nodes.to_crs('epsg:4326').loc[end_node,'geometry']
folium.Circle(location=[end_node.y, end_node.x],color='red').add_to(m)

# add turns
folium.LayerControl().add_to(m)

m

In [ ]:
modeled_results[71].keys()

In [ ]:
minx, miny, maxx, maxy = modeled_summary.to_crs(epsg='4326').total_bounds
x_mean = (maxx - minx) / 2 + minx
y_mean = (maxy - miny) / 2 + miny
print(x_mean,y_mean)


In [ ]:
modeled_results[71]['origin_node']

In [ ]:
matched_summary.columns

In [ ]:
tripid = 71
edges = modeled_results[tripid]['modeled_edges']
list_of_edges = list(zip(edges['linkid'],edges['reverse_link']))
list_of_turns = [(list_of_edges[i][0],list_of_edges[i][1],list_of_edges[i+1][0],list_of_edges[i+1][1]) for i in range(0,len(list_of_edges)-1)]
trip_turns = turns.loc[list_of_turns]

import folium
import folium.vector_layers
#m = folium.Map(tiles="cartodbpositron")
test = modeled_summary.loc[modeled_summary['tripid']==tripid]
#turn types
turn_types = trip_turns.groupby('turn_type')['source_A'].agg(list)

turn_type_colors = {
    'left': 'orange',
    'straight': 'black',
    'right': 'red',
    'u-turn': 'black'
}

minx, miny, maxx, maxy = test.to_crs(epsg='4326').total_bounds
x_mean = (maxx - minx) / 2 + minx
y_mean = (maxy - miny) / 2 + miny
center = [y_mean,x_mean]


In [ ]:
testing = trip_turns[trip_turns['turn_type'].notna()]
testing['geometry'] = nodes.loc[testing['source_B'].tolist(),'geometry'].tolist()
testing = gpd.GeoDataFrame(testing[['source_B','turn_type','signalized','geometry']],crs=nodes.crs)

In [ ]:
m = folium.Map(location=center, zoom_start=14, tiles="cartodbpositron")

folium.GeoJson(test.to_crs(epsg='4326').to_json()).add_to(m)

folium.GeoJson(
    testing.to_crs(epsg='4326').to_json(),
    name="Turn Types",
    tooltip=folium.GeoJsonTooltip(fields=['turn_type']),
    marker=folium.Circle(radius=30, fill_color="orange", fill_opacity=0.5, color="black", weight=0),
    style_function=lambda x: {
        'fillColor':turn_type_colors[x['properties']['turn_type']]
        'fill_opacity': 
    }
).add_to(m)
m

In [ ]:
turn_type, node_list = turn_types.reset_index().values[0]

In [ ]:
m = folium.Map(location=center, zoom_start=14, tiles="cartodbpositron")
folium.GeoJson(
    nodes.loc[node_list].to_crs(epsg='4326').to_json(),
    name=turn_type,
    tooltip=turn_type,
    marker=folium.Circle(radius=30, fill_color="orange", fill_opacity=0.5, color="black", weight=0),
    style_function=lambda x: {
        'fillColor':turn_type_colors[turn_type]
    }
).add_to(m)
m

In [ ]:
nodes.loc[node_list].to_crs(epsg='4326')

In [ ]:
turn_type, node_list = turn_types.reset_index().values[1]

folium.GeoJson(
    nodes.loc[node_list].to_crs(epsg='4326').to_json(),
    name=turn_type,
    tooltip=turn_type,
    marker=folium.Circle(radius=30, fill_color="orange", fill_opacity=0.5, color="black", weight=0),
    style_function=lambda x: {
        'fillColor':turn_type_colors[turn_type]
    }
).add_to(m)

m

In [ ]:
turn_type

In [ ]:
turn_types.reset_index().values

In [ ]:
nodes.loc[node_list,'geometry'].to_crs(epsg='4326').to_json()

In [ ]:

#signals
trip_turns.loc[trip_turns['signalized']==True,'source_A'].tolist()

#unsign_major_road_crossing

trip_turns.columns

#trip_turns['source_B']

In [ ]:
turns

In [ ]:
import folium
import requests

m = folium.Map(tiles="cartodbpositron")

# geojson_data = requests.get(
#     "https://raw.githubusercontent.com/python-visualization/folium-example-data/main/world_countries.json"
# ).json()

folium.GeoJson(
    matched_summary.iloc[0:20].to_crs(epsg='4326').to_json(),
    name="hello world",
    tooltip=folium.GeoJsonTooltip(fields=['tripid'])).add_to(m)

#folium.LayerControl().add_to(m)

m


In [ ]:
# #dicts for referencing certain link attributes quickly
# length_dict = dict(zip(links['linkid'],links['length_ft'])) # need this for loss function
# geo_dict = dict(zip(links['linkid'],links['geometry']))

These did well

In [ ]:
fpr_results = stochastic_optimization.first_preference_recovery(train_set,results_dict,**{'length_dict':length_dict,'overlap_threshold':0.7})
fpr_results

In [ ]:
import random
tripid = random.choice(fpr_results)
tripid
#retrieve chosen path linkids and convert them to tuple
chosen = [tuple(row) for row in train_set[tripid]['matched_edges'].to_numpy()]
shortest = [tuple(row) for row in train_set[tripid]['shortest_edges'].to_numpy()]

#retrieve modeled path linkids
start_node = train_set[tripid]['origin_node']
end_node = train_set[tripid]['destination_node']
modeled_edges = results_dict[(start_node,end_node)]['edge_list']

#get geos (non-directional)
chosen_geo = [geo_dict[linkid[0]] for linkid in chosen]
shortest_geo = [geo_dict[linkid[0]] for linkid in shortest]
modeled_geo = [geo_dict[linkid[0]] for linkid in modeled_edges]

chosen_lines = gpd.GeoSeries(chosen_geo,crs='epsg:2240')
shortest_lines = gpd.GeoSeries(shortest_geo,crs='epsg:2240')
modeled_lines = gpd.GeoSeries(modeled_geo,crs='epsg:2240')

stochastic_optimization.visualize_three_no_legend(chosen_lines,shortest_lines,modeled_lines)

and these not so much

In [ ]:
import random
not_good = list(set(test_set.keys()) - set(fpr_results))

In [ ]:
tripid = random.choice(not_good)
tripid
#retrieve chosen path linkids and convert them to tuple
chosen = [tuple(row) for row in test_set[tripid]['matched_edges'].to_numpy()]
shortest = [tuple(row) for row in test_set[tripid]['shortest_edges'].to_numpy()]

#retrieve modeled path linkids
start_node = test_set[tripid]['origin_node']
end_node = test_set[tripid]['destination_node']
modeled_edges = results_dict[(start_node,end_node)]['edge_list']

#get geos (non-directional)
chosen_geo = [geo_dict[linkid[0]] for linkid in chosen]
shortest_geo = [geo_dict[linkid[0]] for linkid in shortest]
modeled_geo = [geo_dict[linkid[0]] for linkid in modeled_edges]

chosen_lines = gpd.GeoSeries(chosen_geo,crs='epsg:2240')
shortest_lines = gpd.GeoSeries(shortest_geo,crs='epsg:2240')
modeled_lines = gpd.GeoSeries(modeled_geo,crs='epsg:2240')

stochastic_optimization.visualize_three_no_legend(chosen_lines,shortest_lines,modeled_lines)